# Проект по SQL

## Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание таблиц

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Задание на проект

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Практическая часть

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# подключаем к базе данных
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
pd.set_option('display.max_columns', None)

Выведем все таблицы черезе цикл:

In [5]:
tables = ['books', 'authors', 'ratings', 'reviews', 'publishers']
for table in tables:
  print(table)
  query = 'select * from {} limit 5'.format(table)
  x = pd.io.sql.read_sql(query, con = engine) 
  print(x)
  print()

books
   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  

authors
   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4          5        Alan Mo

### Посчитать, сколько книг вышло после 1 января 2000 года;

In [6]:
query = '''SELECT COUNT(book_id)
FROM books
WHERE publication_date > ('2000-01-01')'''
pd.io.sql.read_sql(query, con = engine) 

,count
0,819


### Для каждой книги посчитать количество обзоров и среднюю оценку

In [7]:
query = '''
SELECT  books.book_id AS id, title, ROUND(AVG(rating),2) as rating_average, COUNT(DISTINCT review_id) as reviews_count
FROM 
books
LEFT JOIN ratings on books.book_id = ratings.book_id
LEFT JOIN reviews on books.book_id = reviews.book_id
GROUP BY id
'''
rate_rev = pd.io.sql.read_sql(query, con = engine) 
rate_rev

,id,title,rating_average,reviews_count
0,1,'Salem's Lot,3.67,2
1,2,1 000 Places to See Before You Die,2.50,1
2,3,13 Little Blue Envelopes (Little Blue Envelope...,4.67,3
3,4,1491: New Revelations of the Americas Before C...,4.50,2
4,5,1776,4.00,4
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3.67,3
996,997,Xenocide (Ender's Saga #3),3.40,3
997,998,Year of Wonders,3.20,4
998,999,You Suck (A Love Story #2),4.50,2


In [8]:
rate_rev['reviews_count'].sum()

2793

### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [9]:
query = '''
SELECT publisher, COUNT(book_id) as book_count
FROM 
(SELECT * FROM books
WHERE num_pages > 50) as thick_books
JOIN publishers ON thick_books.publisher_id = publishers.publisher_id
GROUP BY publisher
ORDER BY book_count DESC
LIMIT 2
'''
pd.io.sql.read_sql(query, con = engine) 

,publisher,book_count
0,Penguin Books,42
1,Vintage,31


### Определить автора с самой высокой средней оценкой книг;

In [10]:
query = '''
WITH well_rated_books AS (
SELECT books.book_id AS id,  books.title, books.author_id, COUNT(rating) AS ratings_cnt
FROM books 
LEFT JOIN ratings on books.book_id = ratings.book_id
GROUP BY id
HAVING COUNT(rating) > 50)

SELECT author, AVG(rating) as average_rating
FROM well_rated_books
LEFT JOIN authors on well_rated_books.author_id = authors.author_id
LEFT JOIN ratings on well_rated_books.id = ratings.book_id
GROUP BY author
ORDER BY average_rating DESC
LIMIT 3
'''
pd.io.sql.read_sql(query, con = engine) 

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914


### Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [11]:
query = '''
WITH top_raters AS 
(SELECT username, COUNT(rating) as rating_cnt
FROM ratings
GROUP BY username
HAVING COUNT(rating) > 50), 

review_counts AS (
SELECT top_raters.username AS user, COUNT (review_id) as review_cnt
FROM top_raters
LEFT JOIN reviews on top_raters.username = reviews.username
GROUP BY top_raters.username)

SELECT ROUND(AVG(review_cnt), 2)
FROM review_counts
'''
pd.io.sql.read_sql(query, con = engine) 

,round
0,24.33
